In [11]:
import d2lzh as d2l
from mxnet import nd

"""
多输入通道，单输出通道
"""
def corr2d_multi_in(X, K):
    """
    :param X:
    :param K: 核
    :return: 计算后的多通道图片
    """
    """
    先按照x,k的第0维（通道维）遍历x,k。得到每个通道的图片。
    ---> 卷积层d2l.corr2d(x, k) --->输出一张单通道图片。因为有for循环故会产生多张单通道图片，通过nd.add_n()将他们数值相加。得到一张单通道图片
    """
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])
x = nd.array([[[0,1,2], [3,4,5],[6,7,8]], [[1,2,3], [4,5,6],[7,8,9]]])  # (2,3,3)
k = nd.array([[[0,1],[2,3]],[[1,2], [3,4]]])  # (2,2,2)
corr2d_multi_in(x, k)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

In [3]:
"""
多输出通道
"""
def corr2d_multi_in_out(x, K):
    return nd.stack(*[corr2d_multi_in(x,k) for k in K])  # 通过nd.stack()将多张单通道图片叠加成一张多通道图片
k = nd.stack(k, k+1, k+2)
k.shape

(3, 2, 2, 2)

In [4]:
corr2d_multi_in_out(x, k)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

In [8]:
"""
1x1卷积层
"""
def corr2d_multi_in_out_1x1(x, k):
    c_i, h, w = x.shape
    c_o = k.shape[0]  # 通道数
    x = x.reshape((c_i, h*w))  # 每个通道的图片拉成一行
    k = k.reshape((c_o, c_i))  # 生成c_o行 c_i列权重
    y = nd.dot(k, x)  # 点积
    return y.reshape((c_o, h, w))  # 还原成这种格式

In [9]:
"""
验证
"""
x = nd.random.uniform(shape=(3, 3, 3))
k = nd.random.uniform(shape=(2, 3, 1, 1))

y1 = corr2d_multi_in_out_1x1(x,k)
y2 = corr2d_multi_in_out(x, k)
(y1-y2).norm().asscalar() < 1e-6

True

In [17]:
y1.shape

(2, 3, 3)